# Q2(a)

In [12]:
import pandas as pd
import numpy as np
data = pd.read_csv('DiabetesTraining.csv')
data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,80.0,0,1,never,25.19,6.6,140,0
2,Female,54.0,0,0,No Info,27.32,6.6,80,0
3,Male,28.0,0,0,never,27.32,5.7,158,0
4,Female,36.0,0,0,current,23.45,5.0,155,0


In [38]:
features = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']
label = ['diabetes']
X = data[features].values
Y = data[label].values
X.shape, Y.shape

((1501, 4), (1501, 1))

In [39]:
standardize_X = X / np.std(X, axis=0)
standardize_X

array([[3.4839557 , 3.83445787, 6.11662291, 3.60690783],
       [3.4839557 , 3.83445787, 6.11662291, 3.60690783],
       [2.3516701 , 4.15868952, 6.11662291, 2.06109019],
       ...,
       [3.00491179, 4.15868952, 5.65324239, 2.18990833],
       [2.22102176, 5.7083037 , 5.65324239, 2.31872647],
       [0.30484612, 2.76129678, 5.7459185 , 3.60690783]])

In [50]:
D1 = standardize_X[Y.flatten() == 0]
D2 = standardize_X[Y.flatten() != 0]
D1, D2

(array([[3.4839557 , 3.83445787, 6.11662291, 3.60690783],
        [3.4839557 , 3.83445787, 6.11662291, 3.60690783],
        [2.3516701 , 4.15868952, 6.11662291, 2.06109019],
        ...,
        [3.00491179, 4.15868952, 5.65324239, 2.18990833],
        [2.22102176, 5.7083037 , 5.65324239, 2.31872647],
        [0.30484612, 2.76129678, 5.7459185 , 3.60690783]]),
 array([[ 1.91617564,  2.93939585,  6.02394681,  5.15272548],
        [ 2.9178129 ,  4.15868952,  6.02394681,  5.15272548],
        [ 2.17747232,  4.15868952,  5.28253797,  6.69854312],
        [ 3.17910958,  3.9440573 ,  8.34084943,  4.12218038],
        [ 2.30812065,  4.15868952,  6.48732733,  4.09641676],
        [ 2.17747232,  5.65654841,  8.34084943,  4.09641676],
        [ 2.9178129 ,  9.6630165 ,  8.15549722,  3.99336225],
        [ 2.48231844,  4.15868952,  7.59944059,  3.24621705],
        [ 1.56778007,  4.91218561,  5.7459185 ,  5.66799803],
        [ 2.61296678,  4.15868952,  6.95070786,  7.72908822],
        [ 2.91781

In [73]:
m1 = np.mean(D1, axis=0)
d1 = D1 - m1
w1 = np.dot(d1.T, d1)
m2 = np.mean(D2, axis=0)
d2 = D2 - m2
w2 = np.dot(d2.T, d2)
Sw = w1+w2
Swinv = np.linalg.inv(Sw)
w = Swinv @ (m1 - m2)
print('mean of class 1 is ', m1)
print('mean of class 2 is ', m2)
print('within-class scatter matrix is ')
print(Sw)
print('The value of w is ')
print(w)

mean of class 1 is  [1.73862886 4.04288329 5.0267895  3.42230893]
mean of class 2 is  [2.61151513 4.87052768 6.58154804 4.69864155]
within-class scatter matrix is 
[[1416.87800726  405.5155054    86.38073565   12.62613915]
 [ 405.5155054  1425.3721456    38.40476152  -53.86274044]
 [  86.38073565   38.40476152 1234.11747815  -98.94739238]
 [  12.62613915  -53.86274044  -98.94739238 1321.1452261 ]]
The value of w is 
[-0.00039097 -0.00047507 -0.00130421 -0.00107939]


# Q2(b)

In [99]:
from sklearn.metrics import mutual_info_score
from scipy.stats import entropy

ht = data['hypertension'].values
hd = data['heart_disease'].values
ht, hd, Y

(array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 array([1, 1, 0, ..., 0, 0, 0], dtype=int64),
 array([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]], dtype=int64))

In [125]:
def GI(n1, n2):
    return 1 - (n1/(n1 + n2))**2 - (n2/(n1 + n2))**2
def totalGI(feature, label):
    ht0_Y0 = feature[(feature.flatten() == 0) & (label.flatten() == 0)].size
    ht0_Y1 = feature[(feature.flatten() == 0) & (label.flatten() == 1)].size
    ht1_Y0 = feature[(feature.flatten() == 1) & (label.flatten() == 0)].size
    ht1_Y1 = feature[(feature.flatten() == 1) & (label.flatten() == 1)].size
    ht0_num = ht0_Y0 + ht0_Y1
    ht1_num = ht1_Y0 + ht1_Y1
    GIht = ht0_num/(ht0_num + ht1_num)*GI(ht0_Y0, ht0_Y1) + ht1_num/(ht0_num + ht1_num)*GI(ht1_Y0, ht1_Y1)
    return GIht
def compute_entropy(labels):
    """Compute the entropy of a set of labels."""
    _, counts = np.unique(labels, return_counts=True)
    probabilities = counts / float(len(labels))
    # print(counts)
    return entropy(probabilities, base=2)
def entropy_gain(feature, Y):
    ht0 = Y[feature.flatten() == 0]
    ht1 = Y[feature.flatten() == 1]
    return compute_entropy(Y) - compute_entropy(ht0)*ht0.size/(ht0.size+ht1.size) - compute_entropy(ht1)*ht1.size/(ht0.size+ht1.size)

In [128]:
print('The Gini Impurity of firstly splitting by "hypertension" is ', totalGI(ht, Y))
print('The Gini Impurity of firstly splitting by "heart_disease" is ', totalGI(hd, Y))
print('The information gain of firstly splitting by "hypertension" is ', entropy_gain(ht, Y))
print('The information gain of firstly splitting by "heart_disease" is ', entropy_gain(hd, Y))

The Gini Impurity of firstly splitting by "hypertension" is  0.14295615100629247
The Gini Impurity of firstly splitting by "heart_disease" is  0.14601404503959087
The entropy at the begining is  0.4019913642500085
The information gain of firstly splitting by "hypertension" is  0.014297005653686627
The information gain of firstly splitting by "heart_disease" is  0.003982146278906959
